<a href="https://colab.research.google.com/github/JonathanSum/Pytorch_Lighting_example/blob/master/SimCLR1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pytorch-lightning-bolts

     |████████████████████████████████| 184kB 9.0MB/s 
     |████████████████████████████████| 6.8MB 15.4MB/s 
     |████████████████████████████████| 788kB 38.9MB/s 
     |████████████████████████████████| 409kB 54.9MB/s 
     |████████████████████████████████| 92kB 12.1MB/s 
     |████████████████████████████████| 2.8MB 49.0MB/s 
  Created wheel for test-tube: filename=test_tube-0.7.5-cp36-none-any.whl size=25357 sha256=f71d8c45a1bccb87c0bedc65e6f245b6121e31ad857f42d152f9b890ad807324
  Stored in directory: /root/.cache/pip/wheels/02/52/96/c8b6f3c345cfd3284845ef50818c6996a5658006fe50e40e98
Successfully built test-tube
ERROR: tensorflow 2.3.0 has requirement tensorboard<3,>=2.3.0, but you'll have tensorboard 2.2.0 which is incompatible.
ERROR: pytorch-lightning 0.9.0 has requirement future>=0.17.1, but you'll have future 0.16.0 which is incompatible.
ERROR: pytorch-lightning 0.9.0 has requirement PyYAML>=5.1, but you'll have pyyaml 3.13 which is incompatible.
  Found existing installat

In [ ]:
pip install pytorch-lightning-bolts

In [58]:
from torch.optim import Adam

In [ ]:
import cv2
import numpy as np
import torchvision.transforms as transforms

from typing import Optional
import torch
from torch import nn
from torch.nn import functional as F

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks import LearningRateLogger

from pl_bolts.models.self_supervised.resnets import resnet50
from pl_bolts.optimizers.lr_scheduler import LinearWarmupCosineAnnealingLR
from pl_bolts.datamodules import CIFAR10DataModule, STL10DataModule, ImagenetDataModule
from pl_bolts.metrics import mean, accuracy

from pl_bolts.models.self_supervised.evaluator import Flatten
from pl_bolts.transforms.dataset_normalizations import cifar10_normalization, stl10_normalization
from pl_bolts.transforms.dataset_normalizations import imagenet_normalization
from pl_bolts.optimizers.lars_scheduling import LARSWrapper


In [ ]:
class GaussianBlur(object):
    #implements Gaussian blur as described in the SimCLR paper
    def __init__(self, kernel_size, p=0.5, min=0.1, max=2.0):
        self.min = min
        self.max = max

        #kernel size is set to be 10% of the image height/width
        self.kernel_size = kernel_size
        self.p = p

    def __call__(self, sample):
        sample = np.array(sample)

        # blur the image with a 50% chance
        prob = np.random.random_sample()

        if prop < self.p:
            sigma = (self.max - self.min) * np.random.random_sample() + self.min
            sample = cv2.GaussianBlur(sample, (self.kernel_size, self.kernel_size), sigma)

        return sample

In [ ]:
# class SimCLRTrainDataTransform(object):
#     def __init__(
#         self,
#         input_height: int = 224,
#         gaussian_blur: bool = False,
#         jitter_strength: float = 1.,
#         normalize: Optional[transforms.Normalize] = None
#     ) -> None:

#         self.jitter_strength = jitter_strength
#         self.input_height = input_height
#         self.gaussian_blur = gaussian_blur
#         self.normalize = normalize

#         self.color_jitter = transforms.ColorJitter(
#             0.8 * self.jitter_strength,
#             0.8 * self.jitter_strength,
#             0.8 * self.jitter_strength,
#             0.2 * self.jitter_strength
#         )

#         data_transforms = [
#             transforms.RandomResizedCrop(size=self.input_height),
#             transforms.RandomHorizontalFlip(p=0.5),
#             transforms.RandomApply([self.color_jitter], p=0.8),
#             transforms.RandomGrayscale(p=0.2)
#         ]

#         if self.gaussian_blur:
#             data_transforms.append(GaussianBlur(kernel_size=int(0.1 * self.input_height, p=0.5)))

#         data_transforms.append(transforms.ToTensor())

#         if self.normalize:
#             data_transforms.append(normalize)

#         self.train_transform = transforms.Compose(data_transforms)

#     def __call__(self, sample):
#         transform = self.train_transform

#         xi = transform(sample)
#         xj = transform(sample)

#         return xi, xj


In [31]:
class SimCLREvalDataTransform(object):
    def __init__(
        self,
        input_height: int = 224,
        normalize: Optional[transforms.Normalize] = None
    ):

      self.input_height = input_height
      self.normalize = normalize

      data_transforms = [
          transforms.Resize(self.input_height),
          transforms.ToTensor()
      ]

      if self.normalize:
          data_transforms.append(normalize)
      
      self.test_transform = transforms.Compose(data_transforms) 
    def __call__(self, sample):
        transform = self.test_transform

        xi = transform(sample)
        xj = transform(sample)

        return xi, xj

In [70]:
def nt_xent_loss(out_1, out_2, temperature):
    out = torch.cat([out_1, out_2],  dim=0)
    n_samples = len(out)

    # Full similarity matrix
    cov = torch.mm(out,  out.t().contiguous())
    sim = torch.exp(cov / temperature)

    mask = ~torch.eye(n_samples,  device = sim.device).bool()
    neg = sim.masked_select(mask).view(n_samples, -1).sum(dim=-1)

    # Positive similarity
    pos = torch.exp(torch.sum(out_1 * out_2, dim=-1) / temperature)
    pos = torch.cat([pos,pos], dim=0)

    loss = -torch.log(pos / neg).mean()
    return loss

In [64]:
class Projection(nn.Module):
    def __init__(self,  input_dim = 2048, hidden_dim=2048, output_dim=128):
        super().__init__()
        self.output_dim = output_dim
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim

        self.model = nn.Sequential(
            nn.AdaptiveAvgPool2d ((1, 1)),
            Flatten(),
            nn.Linear(self.input_dim, self.hidden_dim, bias = True),
            nn.BatchNorm1d(self.hidden_dim),
            nn.ReLU(),
            nn.Linear(self.hidden_dim, self.output_dim,bias = False)
        )
        
    def forward(self, x):
        x = self.model(x)
        return F.normalize(x, dim=1)

In [56]:
# class SimCLR(pl.LightningModule):
#         def __init__(self,batch_size,num_samples, world_size = 1, 
#                      warmup_epochs = 10, max_epochs = 100, lars_lr=0.1, lars_eta = 1e-3, 
#                      opt_weight_decay = 1e-6, loss_temperature=0.5):
#             super().__init__()

#             self.save_hyperparameters()

#             #Using save_hyperparameters() to avoid these repeated work in below
#             # self.batch_size = batch_size
#             # self.num_samples = num_....

#             self.nt_xent_loss = nt_xent_loss
#             self.encoder = resnet50()

#             # h -> || -> z
#             self.projection = Projection()

#             # only needed because of CIFAR-10 images will shrink too much 
#             self.encoder.conv1 = nn.Conv2d(
#                 3, 64,
#                 kernel_size = 3,
#                 stride = 1,
#                 padding = 1,
#                 bias = False
#             )

#             self.maxpool = nn.MaxPool2d(kernel_size=1, stride=1)

#         def exclude_from_wt_decay(self, named_params, weight_decay, skip_list=['bias', 'bn']): 
#             params = []
#             excluded_params = []

#             for name, param in named_params:
#                 if not param.requires_grad:
#                     continue
#                 elif any(layer_name in name for layer_name in skip_list):
#                     excluded_params.append(param)
#                 else:
#                     params.append(param)

#             return [
#                 {'params': params, 'weight_decay': weight_decay},
#                 {'params': excluded_params, 'weight_decay': 0.}
#             ]




#         def configure_optimizers(self):
#             # TRICK 1 (USE lars + filter weights)
#             # exclude certain parameters
#             parameters = self.exclude_from_wt_decay(
#                 self.named_parameters(),
#                 weight_decay = self.hparams.opt_weight_decay
#                 # weight_decay = self.hparams.ept_weight_decay
#             )

#             # optimizer = LARS(parameters, lr=self.hparams.lars_lr, eta=self.hparams.lars_eta)
#             optimizer = LARSWrapper(Adam(parameters, lr=self.hparams.lr))

#             # Trick 2 (after each step)
#             self.hparams.warmup_epochs = self.hparams.warmup_epochs * self.train_iters_per_epoch
#             self.hparams.max_epochs = self.hparams.max_epochs * self.train_iters_per_epoch
            
#             linear_warmup_cosine_decay = LinearWarmupCosineAnnealingLR(
#                 optimizer,
#                 warmu_epochs = self.hparams.warmup_epochs,
#                 warmup_start_lr=0,
#                 eta_min=0
#             )
#             scheduler = {
#                 'scheduler': linear_warmup_cosine_decay,
#                 'interval': 'step',
#                 'frequency': 1
#             }
            
#             return [optimizer], [scheduler]
        
#         def training_step(self, batch, batch_idx):
#             loss = self.shared_step(batch, batch_idx)
#             result = pl.TrainResult(minimize=loss)
#             result.log('train_loss', loss, on_epoch = True)
#             return result

#         # if we did the true part in result.log('train_loss', loss, on_epoch=True)
#         # losses = []
#         # for batch in train_dataloader():
#         #     loss = model.training_step(batch, batch_idx)
#         #     losses.append(loss)
#         #     print('train_loss', loss)

#         # Pytorch Lighting has already done this in below for us
#         # model.eval()
#         # with torch.no_grad():
#         #     loss = model.validation_step(batch)
#         #         model.train()

#         def validation_step(self, batch, batch_idx):
#             loss = self.shared_step(batch, batch_idx)

#             result = pl.EvalResult(checkpoint_on=loss)
#             result.log('avg_val_loss', loss)
#             return result

#         def shared_step(self, batch, batch_idx):
#             (img1, img2), y = batch

#             # ENCODE
#             # encode -> representations
#             # (b, 3, 32, 32) -> (b, 2048, 2, 2)
#             h1 = self.encoder(img1)[-1]
#             h2 = self.encoder(img2)[-1]

#             # PROJECT
#             # img -> E -> h -> || -> z
#             # (b, 2048, 2, 2) -> (b, 128)
#             z1 = self.projection(h1)
#             z2 = self.projection(h2)

#             loss = self.nt_xnet_loss(z1, z2, self.hparams.loss_temperature)

#             return loss








In [60]:
class SimCLR(pl.LightningModule):
    def __init__(self,
                 batch_size,
                 num_samples,
                 warmup_epochs=10,
                 lr=1e-4,
                 opt_weight_decay=1e-6,
                 loss_temperature=0.5,
                 **kwargs):
        """
        Args:
            batch_size: the batch size
            num_samples: num samples in the dataset
            warmup_epochs: epochs to warmup the lr for
            lr: the optimizer learning rate
            opt_weight_decay: the optimizer weight decay
            loss_temperature: the loss temperature
        """
        super().__init__()
        self.save_hyperparameters()

        self.nt_xent_loss = nt_xent_loss
        self.encoder = self.init_encoder()

        # h -> || -> z
        self.projection = Projection()

    def init_encoder(self):
        encoder = resnet50_bn(return_all_feature_maps=False)

        # when using cifar10, replace the first conv so image doesn't shrink away
        encoder.conv1 = nn.Conv2d(
            3, 64,
            kernel_size=3,
            stride=1,
            padding=1,
            bias=False
        )
        return encoder

    def exclude_from_wt_decay(self, named_params, weight_decay, skip_list=['bias', 'bn']):
        params = []
        excluded_params = []

        for name, param in named_params:
            if not param.requires_grad:
                continue
            elif any(layer_name in name for layer_name in skip_list):
                excluded_params.append(param)
            else:
                params.append(param)

        return [
            {'params': params, 'weight_decay': weight_decay},
            {'params': excluded_params, 'weight_decay': 0.}
        ]

    def setup(self, stage):
        global_batch_size = self.trainer.world_size * self.hparams.batch_size
        self.train_iters_per_epoch = self.hparams.num_samples // global_batch_size

    def configure_optimizers(self):
        # TRICK 1 (Use lars + filter weights)
        # exclude certain parameters
        parameters = self.exclude_from_wt_decay(
            self.named_parameters(),
            weight_decay=self.hparams.opt_weight_decay
        )

        optimizer = LARSWrapper(Adam(parameters, lr=self.hparams.lr))

        # Trick 2 (after each step)
        self.hparams.warmup_epochs = self.hparams.warmup_epochs * self.train_iters_per_epoch
        max_epochs = self.trainer.max_epochs * self.train_iters_per_epoch

        linear_warmup_cosine_decay = LinearWarmupCosineAnnealingLR(
            optimizer,
            warmup_epochs=self.hparams.warmup_epochs,
            max_epochs=max_epochs,
            warmup_start_lr=0,
            eta_min=0
        )

        scheduler = {
            'scheduler': linear_warmup_cosine_decay,
            'interval': 'step',
            'frequency': 1
        }

        return [optimizer], [scheduler]

    def forward(self, x):
        if isinstance(x, list):
            x = x[0]

        result = self.encoder(x)
        if isinstance(result, list):
            result = result[-1]
        return result

    def training_step(self, batch, batch_idx):
        loss = self.shared_step(batch, batch_idx)

        result = pl.TrainResult(minimize=loss)
        result.log('train_loss', loss, on_epoch=True)
        return result

    def validation_step(self, batch, batch_idx):
        loss = self.shared_step(batch, batch_idx)

        result = pl.EvalResult(checkpoint_on=loss)
        result.log('avg_val_loss', loss)
        return result

    def shared_step(self, batch, batch_idx):
        (img1, img2), y = batch

        # ENCODE
        # encode -> representations
        # (b, 3, 32, 32) -> (b, 2048, 2, 2)
        h1 = self.encoder(img1)
        h2 = self.encoder(img2)

        # the bolts resnets return a list of feature maps
        if isinstance(h1, list):
            h1 = h1[-1]
            h2 = h2[-1]

        # PROJECT
        # img -> E -> h -> || -> z
        # (b, 2048, 2, 2) -> (b, 128)
        z1 = self.projection(h1)
        z2 = self.projection(h2)

        loss = self.nt_xent_loss(z1, z2, self.hparams.loss_temperature)

        return loss

In [27]:
class SimCLRTrainDataTransform(object):
    def __init__(
        self,
        input_height: int = 224,
        gaussian_blur: bool = False,
        jitter_strength: float = 1.,
        normalize: Optional[transforms.Normalize] = None
    ) -> None:

        self.jitter_strength = jitter_strength
        self.input_height = input_height
        self.gaussian_blur = gaussian_blur
        self.normalize = normalize

        #jitter transform
        self.color_jitter = transforms.ColorJitter(
            0.8 * self.jitter_strength,
            0.8 * self.jitter_strength,
            0.8 * self.jitter_strength,
            0.2 * self.jitter_strength
        )

        data_transforms = [
          transforms.RandomResizedCrop(size=self.input_height),
          transforms.RandomHorizontalFlip(p=0.5),
          transforms.RandomApply([self.color_jitter], p = 0.8),
          transforms.RandomGrayscale(p=0.2)
        ]
        if self.gaussian_blur:
            data_transforms.append(GaussianBlur(kernel_size=int(0.1 * self.input_height, p=0.5)))

        data_transforms.append(transforms.ToTensor())

        if self.normalize:
            data_transforms.append(normalize)

        self.train_transform = transforms.Compose(data_transforms)

    def __call__(self, sample):
      transform = self.train_transform
      xi = transform(sample)
      xj = transform(sample)

      return xi, xj 

In [ ]:
# import os

# # callbacks
# lr_logger = LearningRateLogger
# callbacks = [lr_logger]

# cifar_height = 32
# batch_size = 8   #32GB ram use 1024

# dm = CIFAR10DataModule(os.getcwd(), num_workers=8, batch_size=batch_size)
# dm.train_transforms = SimCLRTrainDataTransform(cifar_height)
# dm.val_transforms = SimCLREvalDataTransform(cifar_height)
# dm.test_transforms = SimCLREvalDataTransform(cifar_height)

# # materialize data
# dm.prepare_data()
# dm.setup()

# train_samples = len(dm.train_dataloader()) * batch_size

# # init model
# mode = SimCLR(batch_size, num_samples = train_samples)

# # now train
# trainer = pl.Trainer(callbacks=callbacks, progress_bar_refresh_rate=10, gpus=1, precision=16)   #, sync_batchnorm=True


# # fit!
# trainer.fit(model, dm)

In [62]:
from pl_bolts.models.self_supervised.resnets import resnet50_bn

In [ ]:
import os
from pl_bolts.callbacks.self_supervised import SSLOnlineEvaluator

# init callbacks
def to_device(batch, device):
    (img1, _), y = batch
    img1 = img1.to(device)
    y = y.to(device)
    return img1, y

online_finetuner = SSLOnlineEvaluator(z_dim=2048 * 2 * 2, num_classes = 10)
online_finetuner.to_device = to_device

lr_logger = LearningRateLogger()

callbacks = [online_finetuner, lr_logger]

# pick data
cifar_height = 32
batch_size = 8
num_samples = 32

# init data
dm = CIFAR10DataModule(os.getcwd(), num_workers=0, batch_size=batch_size)
dm.train_transforms = SimCLRTrainDataTransform(cifar_height)
dm.val_transforms = SimCLREvalDataTransform(cifar_height)

# realize the data
dm.prepare_data()
dm.setup()

train_samples = len(dm.train_dataloader())

model = SimCLR(batch_size=batch_size, num_samples=train_samples)
trainer = pl.Trainer(callbacks=callbacks, progress_bar_refresh_rate=10, gpus=1)
trainer.fit(model, dm)

Files already downloaded and verified
Files already downloaded and verified


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning: Could not log computational graph since the `model.example_input_array` attribute is not set or `input_array` was not given
  warnings.warn(*args, **kwargs)

  | Name                 | Type         | Params
------------------------------------------------------
0 | encoder              | ResNet       | 25 M  
1 | projection           | Projection   | 4 M   
2 | non_linear_evaluator | SSLEvaluator | 8 M   


Buffered data was truncated after reaching the output size limit.